notes: I take some inspiration from this notebook to solve some problems https://github.com/DanKorell/nd00333_AZMLND_Optimizing_a_Pipeline_in_Azure-Starter_Files/blob/master/udacity-project.ipynb 

In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="AzureProject")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-242032
Azure region: eastus2
Subscription id: 61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30
Resource group: aml-quickstarts-242032


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = "AzureProject"

# TODO: Create compute cluster
# Use vm_size = " Standard_D2_V2"in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
# verify that cluser does not exist already
try:
    cpu_cluster=ComputeTarget(workspace=ws, name= cpu_cluster_name)
    print("Found existing cluster, use it.")
except ComputeTargetException:
    compute_config=AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
    cpu_cluster= ComputeTarget.create(ws,cpu_cluster_name,compute_config)

cpu_cluster.wait_for_completion(show_output=True)
# print detailed status for the current cluster
print(cpu_cluster.get_status().serialize())
    

Found existing cluster, use it.

Running
{'errors': [], 'creationTime': '2023-09-29T10:25:45.134850+00:00', 'createdBy': {'userObjectId': '458fb77f-2131-4f8c-870c-60adb39e3295', 'userTenantId': '660b3398-b80e-49d2-bc5b-ac1dc93b5254', 'userName': None}, 'modifiedTime': '2023-09-29T10:29:16.188559+00:00', 'state': 'Running', 'vmSize': 'STANDARD_D2S_V3'}


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
{'--C':choice(0.01,0.1,1,10,50,100,500,1000),
'--max_iter':choice(50,100,200,300)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1,slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.',
                     script='train.py',
                     compute_target=cpu_cluster,
                     environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                    hyperparameter_sampling=ps,
                                    policy=policy,
                                    primary_metric_name='Accuracy',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=3,
                                    max_concurrent_runs=3)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run= exp.submit(hyperdrive_config,show_output=True)
RunDetails(hyperdrive_run).show()

hyperdrive_run.get_status()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_5a197d24-87f4-4442-81b0-5360fc2c6d19
Web View: https://ml.azure.com/runs/HD_5a197d24-87f4-4442-81b0-5360fc2c6d19?wsid=/subscriptions/61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30/resourcegroups/aml-quickstarts-242032/workspaces/quick-starts-ws-242032&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-09-29T11:16:04.153279][GENERATOR][INFO]Trying to sample '3' jobs from the hyperparameter space
[2023-09-29T11:16:04.6446250Z][SCHEDULER][INFO]Scheduling job, id='HD_5a197d24-87f4-4442-81b0-5360fc2c6d19_0' 
[2023-09-29T11:16:04.7841643Z][SCHEDULER][INFO]Scheduling job, id='HD_5a197d24-87f4-4442-81b0-5360fc2c6d19_1' 
[2023-09-29T11:16:04.8771752Z][SCHEDULER][INFO]Scheduling job, id='HD_5a197d24-87f4-4442-81b0-5360fc2c6d19_2' 
[2023-09-29T11:16:04.803955][GENERATOR][INFO]Successfully sampled '3' jobs, they will soon be submitted to the execution target.
[2023-09-29T11:16:05.0159569Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_5a197d24-87f4-4442-8

{'runId': 'HD_5a197d24-87f4-4442-81b0-5360fc2c6d19',
 'target': 'AzureProject',
 'status': 'Completed',
 'startTimeUtc': '2023-09-29T11:16:03.392648Z',
 'endTimeUtc': '2023-09-29T11:18:05.990364Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'acf53ba6-4466-4b48-b897-7211dfa8560f',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1040-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.51.0',
  'space_size': '32',
  'score': '0.9166919575113809',
  'best_child_run_id': 'HD_5a197d24-87f4-4442-81b0-5360fc2c6d19_0',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_5a197d24-87f4-4442-81b0-5360fc2c6d19_0'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClient

In [6]:
import joblib
# Get your best run and save the model from that run.
print(hyperdrive_run.get_children_sorted_by_primary_metric(top=0,reverse =False, discard_no_metric=False))
### YOUR CODE HERE ###
best_run=hyperdrive_run.get_best_run_by_primary_metric()
print("Best run metric:", best_run.get_metrics())
print("Run details",best_run.get_details() )
print("Best run file names", best_run.get_file_names())

[{'run_id': 'HD_5a197d24-87f4-4442-81b0-5360fc2c6d19_0', 'hyperparameters': '{"--C": 0.01, "--max_iter": 100}', 'best_primary_metric': 0.9166919575113809, 'status': 'Completed'}, {'run_id': 'HD_5a197d24-87f4-4442-81b0-5360fc2c6d19_1', 'hyperparameters': '{"--C": 1000, "--max_iter": 50}', 'best_primary_metric': 0.9160849772382398, 'status': 'Completed'}, {'run_id': 'HD_5a197d24-87f4-4442-81b0-5360fc2c6d19_2', 'hyperparameters': '{"--C": 1, "--max_iter": 100}', 'best_primary_metric': 0.9151745068285281, 'status': 'Completed'}]
Best run metric: {'Regularization Strength:': 0.01, 'Max iterations:': 100, 'Accuracy': 0.9166919575113809}
Run details {'runId': 'HD_5a197d24-87f4-4442-81b0-5360fc2c6d19_0', 'target': 'AzureProject', 'status': 'Completed', 'startTimeUtc': '2023-09-29T11:16:11.801156Z', 'endTimeUtc': '2023-09-29T11:16:39.286203Z', 'services': {}, 'properties': {'_azureml.ComputeTargetType': 'amlcdsi', 'ContentSnapshotId': 'acf53ba6-4466-4b48-b897-7211dfa8560f', 'ProcessInfoFile': '

In [7]:
best_run.get_file_names()

['logs/azureml/dataprep/0/backgroundProcess.log',
 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log',
 'logs/azureml/dataprep/0/rslex.log.2023-09-29-11',
 'outputs/model.joblib',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/lifecycler/vm-bootstrapper.log',
 'system_logs/metrics_capability/metrics-capability.log',
 'system_logs/snapshot_capability/snapshot-capability.log',
 'user_logs/std_log.txt']

In [8]:
hypr_mdl='hyperdrive_model'
folder='outputs'
path=os.path.join(folder,'model.joblib')
hyperdrive_model=best_run.register_model(model_name=hypr_mdl, model_path=path)

In [9]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds=TabularDatasetFactory.from_delimited_files(['https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'])

In [10]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [13]:
x.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_cellular,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown
0,57,1,0,0,1,5,1,371,1,999,...,1,0,0,0,0,1,0,0,0,0
1,55,1,0,1,0,5,4,285,2,999,...,0,1,0,0,0,0,0,0,0,1
2,33,1,0,0,0,5,5,52,1,999,...,1,0,0,0,1,0,0,0,0,0
3,36,1,0,0,0,6,5,355,4,999,...,0,1,0,0,0,1,0,0,0,0
4,27,1,0,1,0,7,5,189,2,999,...,1,0,0,0,0,1,0,0,0,0


In [14]:
x['y']=y
x.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
0,57,1,0,0,1,5,1,371,1,999,...,0,0,0,0,1,0,0,0,0,0
1,55,1,0,1,0,5,4,285,2,999,...,1,0,0,0,0,0,0,0,1,0
2,33,1,0,0,0,5,5,52,1,999,...,0,0,0,1,0,0,0,0,0,0
3,36,1,0,0,0,6,5,355,4,999,...,1,0,0,0,1,0,0,0,0,0
4,27,1,0,1,0,7,5,189,2,999,...,0,0,0,0,1,0,0,0,0,0


In [25]:
# DW: Save the Pandas dataframe as a CSV file within the training folder
x.to_csv('training/bankmarketing-train.csv', index=False)

In [26]:
for datastore in ws.datastores:
    print(datastore)
print()
print('default datastore:', ws.get_default_datastore())

datastore
workspacefilestore
workspaceblobstore
workspaceworkingdirectory
workspaceartifactstore

default datastore: {
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-d5bf4b21-6feb-4ecc-9a78-920b0fea4ded",
  "account_name": "mlstrg242032",
  "protocol": "https",
  "endpoint": "core.windows.net"
}


In [27]:
datastore=ws.get_default_datastore()

In [28]:
# DW: Upload the training data to the datastore
datastore.upload(src_dir='./training', target_path='bankmarketing',
                 overwrite=True, show_progress=True)

Uploading an estimated of 2 files
Uploading ./training/bankdataset.csv
Uploaded ./training/bankdataset.csv, 1 files out of an estimated total of 2
Uploading ./training/bankmarketing-train.csv
Uploaded ./training/bankmarketing-train.csv, 2 files out of an estimated total of 2
Uploaded 2 files


$AZUREML_DATAREFERENCE_8db1bda7d42143e4aa722a1ca2f5670f

In [29]:
# DW: Create our dataset object as a tabular data set for access during training on remote compute
train_data = TabularDatasetFactory.from_delimited_files(
    path=datastore.path('bankmarketing/bankmarketing-train.csv'))


In [32]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_data,
    label_column_name='y',
    n_cross_validations=5)

In [33]:
# Submit your automl run
run=exp.submit(config=automl_config, show_output=True)


No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
AzureProject,AutoML_3ab334e0-07b9-48b7-8de9-7b05a744b424,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.


2023/09/29 12:00:56 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!



********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in the training data|
+==============================+================================+======================================+
|3692                          |1                               |32950                                 |
+------------------------------+--------------------------------+-----

2023-09-29:12:20:25,863 INFO     [explanation_client.py:334] Using default datastore for uploads


Current status: EngineeredFeatureExplanations. Computation of engineered features completed
Current status: RawFeaturesExplanations. Computation of raw features started
Current status: RawFeaturesExplanations. Computation of raw features completed
Current status: BestRunExplainModel. Best run model explanations completed
********************************************************************************************


In [34]:
# Retrieve and save your best automl model.

best_run, fitted_model=run.get_output()
print(best_run)
print(fitted_model)

print("Best run metrics :", best_run.get_metrics())
print("Best accuracy run metrics ",best_run.get_metrics(name='accuracy'))
print("Best run details :", best_run.get_details())

Run(Experiment: AzureProject,
Id: AutoML_3ab334e0-07b9-48b7-8de9-7b05a744b424_20,
Type: None,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('11', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=0.6, eta=0.3, gamma=0, max_depth=6, max_leaves=0, n_estimators=10, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_s

In [37]:
#Next, identify the file names for the best AutoML run
best_run.get_file_names()

['accuracy_table',
 'confusion_matrix',
 'explanation/4750b4ac/classes.interpret.json',
 'explanation/4750b4ac/eval_data_viz.interpret.json',
 'explanation/4750b4ac/expected_values.interpret.json',
 'explanation/4750b4ac/features.interpret.json',
 'explanation/4750b4ac/global_names/0.interpret.json',
 'explanation/4750b4ac/global_rank/0.interpret.json',
 'explanation/4750b4ac/global_values/0.interpret.json',
 'explanation/4750b4ac/local_importance_values.interpret.json',
 'explanation/4750b4ac/per_class_names/0.interpret.json',
 'explanation/4750b4ac/per_class_rank/0.interpret.json',
 'explanation/4750b4ac/per_class_values/0.interpret.json',
 'explanation/4750b4ac/rich_metadata.interpret.json',
 'explanation/4750b4ac/true_ys_viz.interpret.json',
 'explanation/4750b4ac/visualization_dict.interpret.json',
 'explanation/4750b4ac/ys_pred_proba_viz.interpret.json',
 'explanation/4750b4ac/ys_pred_viz.interpret.json',
 'explanation/915e8c4a/classes.interpret.json',
 'explanation/915e8c4a/expe

In [ ]:
# DW: Finally, register the model of the best AutoML run to save it
automl_model = best_run.register_model(model_name = 'automl_model',
                                              model_path = 'outputs/model.joblib')

In [ ]:
# End the experiment logging that was started in the first code cell
run.complete()

# Finally, delete the CPU cluster
cpu_cluster.delete()